# Exercícios Spark SQL

In [ ]:
# If necessary, set SPARK_HOME env variable
%env SPARK_HOME=/opt/spark-2.4.3
%env PYSPARK_PYTHON=/usr/bin/python3
#PYSPARK_DRIVER_PYTHON

import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

In [ ]:
from pyspark.sql import functions as SF

In [ ]:
#df = spark.read.csv('s3://<bucket_name>/BASE_JUR001.csv.gz',sep='\t',encoding='utf-8', header=True,inferSchema=True)

In [ ]:
#Download file
!wget --no-check-certificate --no-proxy 'https://s3.us-east-2.amazonaws.com/cesarschool-data-samples/BASE_JUR001.csv.gz'

In [ ]:
df = spark.read.csv('BASE_JUR001.csv.gz',sep='\t',encoding='utf-8', header=True,inferSchema=True)

In [ ]:
df.printSchema()

## Questão 1

Calcular a idade da empresa em anos considerando o REF_DATE.

In [ ]:
df = df.withColumn('AGE', (SF.unix_timestamp('REF_DATE') - SF.unix_timestamp('ABERTURA')) / (365 * 24 * 60 * 60))
df.select(['CNPJ','ABERTURA','REF_DATE','AGE']).show()

## Questão 2

Criar flag ('S','N') indicando se o segundo telefone foi informado ou não.

In [ ]:
df = df.withColumn('FLAG_SEGUNDO_TELEFONE', SF.when(SF.col('TELEFONE_2').isNull(),'N').otherwise('S'))
df.select(['CNPJ','TELEFONE_2','FLAG_SEGUNDO_TELEFONE']).show()

## Questão 3

Remover pontuação do CNPJ_FORMATADO

In [ ]:
@SF.udf('string')
def format_cnpj(cnpj):
    return cnpj.replace('.','').replace('/','').replace('-','')

In [ ]:
df = df.withColumn('CNPJ_WITHOUT_PONCT', format_cnpj(SF.col('CNPJ_FORMATADO')))
df.select(['CNPJ','CNPJ_WITHOUT_PONCT','CNPJ_FORMATADO']).show()

## Questão 4

Excluir colunas que tenham um percentual de valores nulos acima de x%.

In [ ]:
def drop_null_columns(df_, perc):
    count = float(df_.count())
    for col in df_.columns:
        null_count = df_.filter(SF.col(col).isNull()).count()
        if null_count/count > perc:
            df_ = df_.drop(col)
        df_ = df_.cache()
    return df_        

In [ ]:
df_null = drop_null_columns(df, 0.7)
len(df_null.columns)

## Questão 5

Criar duas novas colunas, uma com o ano e outra com o mês da data de abertura da empresa.

In [ ]:
df = df.withColumn('YEAR_FUNDATION', SF.year('ABERTURA').cast('int'))
df = df.withColumn('MONTH_FUNDATION', SF.month('ABERTURA').cast('int'))
df.select(['CNPJ','ABERTURA','YEAR_FUNDATION','MONTH_FUNDATION']).show()

## Questão 6

Criar uma flag indicando se a Atividade Principal da empresa é correspondente a Atividade Principal de todas as empresas do estado em que a empresa tá localizada.

In [ ]:
# Disconsidera as informaçẽos dos sócios
df_aux = df.select('CNPJ','UF','ATIVIDADE_PRINCIPAL').dropDuplicates()
df_aux.printSchema()

In [ ]:
# Conta quantas empresas tem a mesma atividade principal para cada estado
df_aux_2 = df_aux.groupBy(['UF','ATIVIDADE_PRINCIPAL']).agg(SF.count('CNPJ').alias('count'))
df_aux_2.show(5)

In [ ]:
# Encontra a quantidade da atividade principal que tem mais ocorrência
df_aux_3 = df_aux_2.groupBy('UF').agg(SF.max("count").alias('max_count')).withColumnRenamed('UF','UF_A')
df_aux_3.show(5)

In [ ]:
# Descobre a atividade principal que possui essa quantidade
df_aux_4 = df_aux_2.join(df_aux_3,(df_aux_3['UF_A']==df_aux_2['UF'])&(df_aux_3['max_count']==df_aux_2['count']), 'left')
df_aux_4.show(5)

In [ ]:
df_aux_5 = df_aux_4.filter(SF.col('UF_A').isNotNull()).select('UF','ATIVIDADE_PRINCIPAL')\
            .withColumnRenamed('ATIVIDADE_PRINCIPAL','A_P').withColumnRenamed('UF','UF_')
df_aux_5.show(5)

In [ ]:
df_aux = df_aux.join(df_aux_5,(df_aux['UF']==df_aux_5['UF_'])&
                     (df_aux['ATIVIDADE_PRINCIPAL']==df_aux_5['A_P']),'left')
df_aux.show(5)

In [ ]:
df_aux = df_aux.withColumn('FLAG_ATIVIDADE_PRINCIPAL_UF',SF.when(SF.col('A_P').isNull(),'N').otherwise('S'))\
    .drop('A_P').drop('UF_')
df_aux.show()